# Notebook to make figures for conferences and manuscripts

Rainey Aberle

2022/2023

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import contextily as cx
import geopandas as gpd
import pandas as pd
from skimage.measure import find_contours
import ee
import sys
from shapely.geometry import Point, LineString
import rasterio as rio
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, LightSource
import matplotlib
import glob
import wxee as wx
import matplotlib
import pickle
from scipy.signal import medfilt
import os
import glob
import operator
import json
from ast import literal_eval

# path to snow-cover-mapping/
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'

# path to study-sites/
study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/study-sites/'

# determine whether to save output figures
save_figures = True

# path for saving output figures
out_path = base_path+'figures/'

# add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

# load dataset dictionary
dataset_dict = json.load(open(base_path + 'inputs-outputs/datasets_characteristics.json'))

### Define some colormaps

In [ ]:
# -----Imagery Datasets
color_Landsat = '#ff7f00'
color_Sentinel2 = '#984ea3'
color_PlanetScope = '#4daf4a'

ListedColormap([color_Landsat, color_Sentinel2, color_PlanetScope])

In [ ]:
# -----Classified images
# Indicies: 0 = snow, 1 = shadowed snow, 2 = ice, 3 = bare ground, 4 = water
colors_classified = list(dataset_dict['classified_image']['class_colors'].values())
ListedColormap(colors_classified)

## Figure 1. Spectral signatures for earth materials and satellite band ranges

In [ ]:
from matplotlib.patches import Rectangle

# -----Set up figure
# define colors for different materials
color_snow = colors_classified[0]
# color_ice = colors_classified[2]
color_veg = '#006d2c'
color_rock = colors_classified[3]
color_water = colors_classified[4]
colors = [color_snow, color_veg, color_rock, color_water]
# plot
fig, ax = plt.subplots(1, 1, figsize=(12,6))
plt.rcParams.update({'font.size':13})
plt.xlabel('Wavelength [$\mu$m]')
plt.ylabel('Reflectance')
    
# -----Plot satellite band ranges
def draw_boxes(ax, band_ranges, NDSI_indices, y0=0.2, box_height=0.04, 
               facecolor='#bdbdbd', edgecolor='k', alpha=1.0, NDSI_label=False):
    labeled = False
    # loop over band ranges
    for i, band_range in enumerate(band_ranges):
        # convert from nanometers to micrometers
        x0, x1 = band_range[0], band_range[1]
        # calculate width
        box_width = x1-x0
        # create rectangle and add to axes
        ax.add_patch(Rectangle((x0, y0), width=box_width, height=box_height, 
                               facecolor=facecolor, edgecolor=edgecolor, alpha=alpha))
        # plot star on NDSI bands
        if i in NDSI_indices:
            if (not labeled) and NDSI_label:
                label = 'NDSI bands'
                labeled = True
            else:
                label='_nolegend_'
            ax.plot(x0 + box_width/2, y0 + box_height/2, '*k', markersize=10, label=label)

    # add one rectangle to contain all bands
    x0, x1 = band_ranges[0][0], band_ranges[-1][-1]
    box_width = x1-x0
    ax.add_patch(Rectangle((x0, y0), width=box_width, height=box_height, facecolor='none', edgecolor='k', alpha=1.0))
    
    return

# -----Add satellite band ranges
# Landsat 8/9 OLI
L_band_ranges = [[0.45, 0.51], [0.53, 0.59], [0.64, 0.67], [0.85, 0.88], # 2, 3, 4, 5
                 [1.57, 1.65], [2.11, 2.29]] # 6, 7
L_band_names = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']#, 'TIRS1', 'TIRS2']
L_NDSI_band_indices = [1, 4]
draw_boxes(ax, L_band_ranges, L_NDSI_band_indices, y0=0.71, NDSI_label=True, facecolor=color_Landsat)
ax.text(2.32, 0.715, 'Landsat 8/9')
# Sentinel-2 MSI
S2_20_band_ranges = [[0.69, 0.718], [0.727, 0.755], [0.764, 0.802], # B5, B6, B7
                     [0.845, 0.85], [1.52, 1.70], [2.010, 2.37]] # B8A, B11 (SWIR1), B12 (SWIR2)
S2_20_NDSI_band_indices = [4]
draw_boxes(ax, S2_20_band_ranges, S2_20_NDSI_band_indices, y0=0.81, facecolor=color_Sentinel2)
ax.text(2.4, 0.815, 'Sentinel-2 (20m)')

S2_10_band_ranges = [[0.425, 0.555], [0.525, 0.595], [0.635, 0.695], # B2 B3 B4 
                     [0.728, 1.038]] # B8 (NIR)
S2_10_NDSI_band_indices = [1]
draw_boxes(ax, S2_10_band_ranges, S2_10_NDSI_band_indices, y0=0.91, facecolor=color_Sentinel2)
ax.text(1.068, 0.915, 'Sentinel-2 (10m)')

# PlanetScope 4-band
PS_band_ranges = [[0.455, 0.515], [0.51, 0.59], [0.590, 0.670], [0.780, 0.860]]
PS_NDSI_indices = [1, 3]
draw_boxes(ax, PS_band_ranges, PS_NDSI_indices, y0=1.01, facecolor=color_PlanetScope)
ax.text(0.90, 1.015, 'PlanetScope 4-band')

# -----Load spectral signatures data and plot
spec_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/write-ups/snow_cover_mapping_methods_manuscript/figures/USGS_spectral_signatures/'
os.chdir(spec_path)
# define prefixes used in file names for each material
prefixes = ['Melting_snow', 'Aspen', 'Basalt', 'Seawater']
# define labels for plot
labels = ['melting snow', 'vegetation', 'soil', 'seawater']
# loop through prefixes
for i, prefix in enumerate(prefixes):
    # grab folder name
    folder = glob.glob('*'+prefix+'*')[0]
    # load wavelengths
    wave_fn = glob.glob(folder + '/*Wavelengths*.txt')[0]
    wave = pd.read_csv(wave_fn)
    wave = wave[wave.keys()[0]].values
    if prefix=='Basalt':
        refl_fn = glob.glob(folder + '/*'+prefix+'*.txt')[1]
    else:
        refl_fn = glob.glob(folder + '/*'+prefix+'*.txt')[0]
    refl = pd.read_csv(refl_fn)
    refl = refl[refl.keys()[0]].values
    refl[refl<0] = np.nan
    # plot
    ax.plot(wave, refl, '-', color=colors[i], linewidth=3, label=labels[i])
    
ax.grid()
ax.set_xlim(0, 3.3)
ax.set_ylim(0, 1.1)
ax.legend(loc='center right', bbox_to_anchor=[0.8, 0.3, 0.2, 0.2])
plt.show()

# -----Save figure to file
fig.tight_layout()
fig_fn = 'spectral_signatures_satellite_bands.png'
fig.savefig(out_path + fig_fn, facecolor='w', dpi=300)
print('figure saved to file: '+out_path+fig_fn)

## Figure 2. Study sites - USGS Benchmark Glaciers

In [ ]:
# -----Load RGI data
# path to RGI data
rgi_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/GIS_data/RGI/'
# RGI shapefile names
rgi_fns = ['01_rgi60_Alaska/01_rgi60_Alaska.shp', 
           '02_rgi60_WesternCanadaUS/02_rgi60_WesternCanadaUS.shp']
# load and combine rgis
rgis = gpd.GeoDataFrame()
for rgi_fn in rgi_fns:
    rgi = gpd.read_file(rgi_path + rgi_fn)
    rgis = pd.concat([rgis, rgi])
rgis.reset_index(drop=True, inplace=True)

In [ ]:
# -----Count number of glaciers with areas < 1000 km^2 (for manuscript)
print('Total # of glaciers in RGI regions 1 and 2 = ', len(rgis))
print('Number of glaciers with areas < 1 km^2 = ', len(rgis.loc[rgis['Area'] < 1]))
print('Percentage of glaciers with areas < 1 km^2 = ', len(rgis.loc[rgis['Area'] < 1]) / len(rgis))


In [ ]:
# -----Define site specifics
site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry']
site_names_display = ['Wolverine', 'Gulkana', 'Lemon Creek', 'South Cascade', 'Sperry']
site_colors = ['#1f78b4', '#33a02c', '#fec44f', '#cc4c02', '#984ea3']
text_labels = ['a)', 'b)', 'c)', 'd)', 'e)']

# -----Define colormap for elevations
cmap_elev = plt.cm.terrain(np.linspace(0, 1, 100))
cmap_elev = ListedColormap(cmap_elev[25:, :])

# ----Function for formatting contour labels
def fmt(x):
    s = f"{x:.1f}"
    if s.endswith("0"):
        s = f"{x:.0f}"
    return rf"{s} m" if plt.rcParams["text.usetex"] else f"{s} m"

# -----Set up figure
fig, ax = plt.subplots(2, 3, figsize=(16, 12), layout='constrained')
plt.rcParams.update({'font.size':18, 'font.sans-serif':'Arial'})
ax = ax.flatten()
epsg_A = 32610

# -----Loop through sites
i=0
for site_name, site_color, site_name_display, text_label in list(zip(site_names, site_colors, site_names_display, text_labels)):
    ### AOI
    # load file
    AOI_fn = glob.glob(study_sites_path + site_name + '/AOIs/' + site_name + '_USGS_*.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    AOI_WGS = AOI.to_crs(4326)
    # solve for optimal UTM zone
    AOI_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                    AOI_WGS.geometry[0].centroid.xy[1][0]]
    epsg_UTM = f.convert_wgs_to_utm(AOI_centroid[0], AOI_centroid[1])
    # reproject
    AOI_UTM = AOI_WGS.to_crs(epsg_UTM)
    AOI_A = AOI.to_crs(epsg_A)
    ### DEM
    # DEM_fn = glob.glob(study_sites_path + site_name + '/DEMs/' + site_name + '*_clip.tif')[0]
    # DEM = xr.open_dataset(DEM_fn)
    # DEM = DEM.rename({'band_data': 'elevation'})
    # # reproject 
    # DEM = DEM.rio.reproject(str('EPSG:'+epsg_UTM))
    # create meshgrid of coordinates
    X_mesh, Y_mesh = np.meshgrid(DEM.x.data, DEM.y.data)
    ### Plot
    # A) Study sites map
    ax[0].plot(AOI_A.geometry[0].centroid.xy[0][0], AOI_A.geometry[0].centroid.xy[1][0], 
            '.', markerfacecolor=site_color, markeredgecolor='k', markersize=5)
    ax[0].text(AOI_A.geometry[0].centroid.xy[0][0], AOI_A.geometry[0].centroid.xy[1][0],
               text_labels[i], bbox=dict(facecolor='white', edgecolor='black', pad=3))
    # Individual glacier plot
    AOI_UTM.plot(ax=ax[i+1], edgecolor='k', facecolor='none', linewidth=2)
    # CS = ax[i+1].contour(X_mesh, Y_mesh, DEM.elevation.data[0], levels=4, colors='grey')
    # ax[i+1].clabel(CS, CS.levels, inline=True, fmt=fmt, fontsize=10, colors='grey')
    if AOI.geometry[0].geom_type=='MultiPolygon':
        xmin_AOI = np.min([np.min(geom.exterior.coords.xy[0]) for geom in AOI.geometry[0].geoms])
        xmax_AOI = np.max([np.max(geom.exterior.coords.xy[0]) for geom in AOI.geometry[0].geoms])
        ymin_AOI = np.min([np.min(geom.exterior.coords.xy[1]) for geom in AOI.geometry[0].geoms])
        ymax_AOI = np.max([np.max(geom.exterior.coords.xy[1]) for geom in AOI.geometry[0].geoms])      
    else:
        xmin_AOI = np.min(AOI.geometry[0].exterior.coords.xy[0])
        xmax_AOI = np.max(AOI.geometry[0].exterior.coords.xy[0])
        ymin_AOI = np.min(AOI.geometry[0].exterior.coords.xy[1])
        ymax_AOI = np.max(AOI.geometry[0].exterior.coords.xy[1])  
    xmin = xmin_AOI - 0.1*(xmax_AOI - xmin_AOI)
    xmax = xmax_AOI + 0.1*(xmax_AOI - xmin_AOI)
    ymin = ymin_AOI - 0.1*(ymax_AOI - ymin_AOI)
    ymax = ymax_AOI + 0.1*(ymax_AOI - ymin_AOI) 
    # change x and y tick labels to km
    ax[i+1].set_xlim(xmin, xmax)
    ax[i+1].set_ylim(ymin, ymax)
    if i < 3:
        ax[i+1].set_xticks(np.arange(np.round(xmin,-3), np.round(xmax,-3), 2e3))
        ax[i+1].set_yticks(np.arange(np.round(ymin,-3), np.round(ymax,-3), 2e3)) 
    else:
        ax[i+1].set_xticks(np.arange(np.round(xmin,-3), np.round(xmax,-3), 1e3))
        ax[i+1].set_yticks(np.arange(np.round(ymin,-3), np.round(ymax,-3), 1e3)) 
    ax[i+1].set_xticklabels([str(int(x/1e3)) for x in ax[i+1].get_xticks()])
    ax[i+1].set_yticklabels([str(int(y/1e3)) for y in ax[i+1].get_yticks()])
    ax[i+1].set_title(text_label + ' ' + site_name_display + ' Glacier')
    ax[i+1].grid()
    # add axes labels
    if (i==1) or (i==3):
        ax[i].set_ylabel('Northing [km]')
    if i > 1:
        ax[i+1].set_xlabel('Easting [km]')
    cx.add_basemap(ax[i+1], crs='EPSG:'+str(epsg_UTM), source=cx.providers.Esri.WorldImagery, attribution=False)

    # increase loop counter
    i+=1

# A: study sites map
ax[0].set_xlim(-2000000, 1500000)
ax[0].set_ylim(5000000, 7800000)
ax[0].set_xticks([])
ax[0].set_yticks([])
cx.add_basemap(ax[0], crs='EPSG:'+str(epsg_A), source=cx.providers.Esri.WorldGrayCanvas, attribution=False)
rgis_reproj = rgis.to_crs('EPSG:'+str(epsg_A))
rgis_reproj.plot(ax=ax[0], facecolor=colors_classified[2], edgecolor=colors_classified[2])
# fig.colorbar(DEM_im, ax=[ax[2], ax[5]], shrink=0.5, label='Elevation [m]')
plt.show()

if save_figures:
    fig.savefig(out_path+'study_sites.png', dpi=300, facecolor='white', edgecolor='none')
    print('figure saved to file')

## Figure 3. Methods workflow

In [ ]:
font_size = 32
save_figures = 1

# -----Load dataset dictionary
with open(base_path + 'inputs-outputs/datasets_characteristics.json') as fn:
    dataset_dict = json.load(fn)
dataset = 'PlanetScope'

# -----Image settings
# site name
site_name = 'SouthCascade'
# create colormap for classified image
cmp = ListedColormap(colors_classified)

# -----Load AOI as gpd.GeoDataFrame
AOI_fn = study_sites_path + site_name + '/AOIs/' + site_name + '_USGS_*.shp'
AOI_fn = glob.glob(AOI_fn)[0]
AOI = gpd.read_file(AOI_fn)
# reproject the AOI to WGS to solve for the optimal UTM zone
AOI_WGS = AOI.to_crs(4326)
AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                    AOI_WGS.geometry[0].centroid.xy[1][0]]
epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])
# reproject AOI to UTM
AOI_UTM = AOI.to_crs(str(epsg_UTM))

# -----Load DEM as Xarray DataSet
DEM_fn = study_sites_path + site_name + '/DEMs/' + site_name + '*_DEM*.tif'
# load DEM as xarray DataSet
DEM_fn = glob.glob(DEM_fn)[0]
DEM = xr.open_dataset(DEM_fn)
DEM = DEM.rename({'band_data': 'elevation'})
# reproject the DEM to the optimal UTM zone
DEM = DEM.rio.reproject(str('EPSG:'+epsg_UTM))
# remove unnecessary data (possible extra bands from ArcticDEM or other DEM)
if len(np.shape(DEM.elevation.data))>2:
    DEM['elevation'] = DEM.elevation[0]
    
# -----1. Raw image
im_path = study_sites_path + site_name + '/imagery/PlanetScope/mosaics/'
im_fn = '20210924_18.tif'
im = xr.open_dataset(im_path + im_fn)
# determine image date from image mosaic file name
im_date = im_fn[0:4] + '-' + im_fn[4:6] + '-' + im_fn[6:8] + 'T' + im_fn[9:11] + ':00:00'
im_dt = np.datetime64(im_date)
xmin, xmax, ymin, ymax = np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)
# plot
fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.imshow(np.dstack([im.band_data.data[2]/1e4, im.band_data.data[1]/1e4, im.band_data.data[0]/1e4]), 
           extent=(xmin, xmax, ymin, ymax))
AOI.plot(ax=ax1, facecolor='none', edgecolor='k', linewidth=3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.axis('off')

# -----2. Adjusted image
polygon_top, polygon_bottom = f.create_aoi_elev_polys(AOI_UTM, DEM)
im_adj, im_adj_method = f.planetscope_adjust_image_radiometry(im, im_dt, polygon_top, polygon_bottom, dataset_dict, skip_clipped=False)
# plot
fig2, ax2 = plt.subplots(figsize=(8,8))
ax2.imshow(np.dstack([im_adj.Red.data[0], im_adj.Green.data[0], im_adj.Blue.data[0]]), 
           extent=(xmin, xmax, ymin, ymax))
AOI_UTM.plot(ax=ax2, facecolor='none', edgecolor='k', linewidth=3)
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(ymin, ymax)
ax2.axis('off')

# -----3. Classified image
im_classified_path = study_sites_path + site_name + '/imagery/classified/'
im_classified_fn = '20210924T180000_SouthCascade_PlanetScope_classified.nc'
im_classified = xr.open_dataset(im_classified_path + im_classified_fn)
# remove no data values
im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
# plot
fig3, ax3 = plt.subplots(figsize=(8,8))
plt.rcParams.update({'font.size':font_size, 'font.sans-serif':'Arial'})
ax3.imshow(im_classified.classified.data[0], cmap=cmp, vmin=1, vmax=5,
           extent=(xmin, xmax, ymin, ymax))
AOI.plot(ax=ax3, facecolor='none', edgecolor='k', linewidth=3)
# plot dummy points for legend
ax3.scatter(0, 0, marker='s', color=colors_classified[0], s=300, label='snow')
ax3.scatter(0, 0, marker='s', color=colors_classified[1], s=300, label='shadowed snow')
ax3.scatter(0, 0, marker='s', color=colors_classified[2], s=300, label='ice')
ax3.scatter(0, 0, marker='s', color=colors_classified[3], s=300, label='rock')
ax3.scatter(0, 0, marker='s', color=colors_classified[4], s=300, label='water')
ax3.set_xlim(xmin, xmax+300)
ax3.set_ylim(ymin, ymax)
ax3.legend(loc='center right', bbox_to_anchor=[1.3, 0.7, 0.2, 0.2])
ax3.axis('off')

# -----4. Snow edges
# create binary snow matrix
im_binary = xr.where(im_classified.classified.data[0] <=2, 1, 0)
# Find contours at a constant value of 0.5 (between 0 and 1)
contours = find_contours(im_binary, 0.5)
# convert contour points to image coordinates
contours_coords = []
for contour in contours: 
    ix = np.round(contour[:,1]).astype(int)
    iy = np.round(contour[:,0]).astype(int)
    coords = (im_adj.isel(x=ix, y=iy).x.data, # image x coordinates
              im_adj.isel(x=ix, y=iy).y.data) # image y coordinates
    # zip points together
    xy = list(zip([x for x in coords[0]], 
                  [y for y in coords[1]]))
    contours_coords = contours_coords + [xy]
# plot
fig4, ax4 = plt.subplots(figsize=(8,8))
plt.rcParams.update({'font.size':font_size, 'font.sans-serif':'Arial'})
binary_plt = ax4.imshow(im_binary, cmap='Greys')
for i, contour in list(zip(np.arange(0,len(contours)), contours)):
    if i==0:
        plt.plot(contour[:,1], contour[:,0], '-m', label='edges', linewidth=2)
    else:
        plt.plot(contour[:,1], contour[:,0], '-m', label='_nolegend', linewidth=2)
# plot dummy points for legend
ax4.scatter(np.array([-10, -9]),np.array([-10, -9]), edgecolor='k', facecolor='k', s=100, label='snow')
ax4.scatter(np.array([-10, -9]),np.array([-10, -9]), edgecolor='k', facecolor='w', s=100, label='no snow')
ax4.set_xlim(0,len(im.x.data)+300)
ax4.set_ylim(len(im.y.data), 0)
ax4.legend(loc='upper right', bbox_to_anchor=[0.9, 0.8, 0.2, 0.2])
ax4.axis('off')

# -----5. Snow line
snowlines_fn = study_sites_path + site_name + '/imagery/snowlines/20210924T180000_SouthCascade_PlanetScope_snowline.csv'
snowlines = pd.read_csv(snowlines_fn)
snowlines_X = snowlines.snowlines_coords_X.apply(literal_eval)[0]
snowlines_Y = snowlines.snowlines_coords_Y.apply(literal_eval)[0]

# plot
fig5, ax5 = plt.subplots(figsize=(8,8))
plt.rcParams.update({'font.size':font_size, 'font.sans-serif':'Arial'})
binary_plt = ax5.imshow(im_binary, 
                        extent=(xmin, xmax, ymin, ymax),
                        cmap='Greys')
ax5.plot(snowlines_X, snowlines_Y, '.c', label='_nolegend', markersize=10)
ax5.plot(-20, -20, 'c', label='snowline', linewidth=5)
# plot dummy points for legend
ax5.scatter(np.array([-10, -9]),np.array([-10, -9]), edgecolor='k', facecolor='k', s=100, label='snow')
ax5.scatter(np.array([-10, -9]),np.array([-10, -9]), edgecolor='k', facecolor='w', s=100, label='no snow')
ax5.set_xlim(xmin, xmax+300)
ax5.set_ylim(ymin, ymax)
ax5.legend(loc='center right', bbox_to_anchor=[1.0, 0.6, 0.2, 0.2])
ax5.axis('off')
plt.show()

if save_figures:
    fig1.savefig(out_path+'methods_workflow_1.png', dpi=300, facecolor='white', edgecolor='none', bbox_inches='tight')
    fig2.savefig(out_path+'methods_workflow_2.png', dpi=300, facecolor='white', edgecolor='none', bbox_inches='tight')
    fig3.savefig(out_path+'methods_workflow_3.png', dpi=300, facecolor='white', edgecolor='none', bbox_inches='tight')
    fig4.savefig(out_path+'methods_workflow_4.png', dpi=300, facecolor='white', edgecolor='none', bbox_inches='tight')
    fig5.savefig(out_path+'methods_workflow_5.png', dpi=300, facecolor='white', edgecolor='none', bbox_inches='tight')
    print('figures saved to file')

## Figure 4. Images used for classification performance assessment

In [ ]:
data_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/classified-points/assessment/'

# grab Sentinel-2_SR image file names
os.chdir(data_path)
im_fns = sorted(glob.glob('*Sentinel-2_SR*.tif'))
# plot Lemon Creek images on top
im_fns = im_fns[2:] + im_fns[0:2]

# grab validation point names
data_pts_fns = sorted(glob.glob('*.shp'))

# set up figure
fig, ax = plt.subplots(2, 2, figsize=(8,8))
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
ax = ax.flatten()
text_labels = ['a)', 'b)', 'c)', 'd)']
# plot dummy points for legend
ax[0].plot(0,0, '.', markersize=8, color=colors_classified[0], label='snow')
ax[0].plot(0,0, '.', markersize=8, color=colors_classified[3], label='no snow')

# loop through image files
for i, im_fn in enumerate(im_fns):
    
    print(im_fn)
    
    # open image and plot
    im = rxr.open_rasterio(im_fn)
    im = im / 1e4
    ax[i].imshow(np.dstack([im.data[3], im.data[2], im.data[1]]),
                extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
    
    # load data points and plot
    site_name = im_fn.split('_')[0]
    im_date = im_fn[-12:-4]
    data_pts_snow_fn = [x for x in data_pts_fns if (site_name in x) and (im_date[0:6] in x) and ('_snow' in x)]
    if len(data_pts_snow_fn) > 0:
        data_pts_snow = gpd.read_file(data_pts_snow_fn[0])
        data_pts_snow = data_pts_snow.to_crs(im.rio.crs)
        data_pts_snow.plot(ax=ax[i], color=colors_classified[0], markersize=1)
    data_pts_no_snow_fn = [x for x in data_pts_fns if (site_name in x) and (im_date[0:6] in x) and ('no-snow' in x)]
    if len(data_pts_no_snow_fn) > 0:
        data_pts_no_snow = gpd.read_file(data_pts_no_snow_fn[0])
        data_pts_no_snow = data_pts_no_snow.to_crs(im.rio.crs)
        data_pts_no_snow.plot(ax=ax[i], color=colors_classified[3], markersize=1)
        
    # set axis limits and ticks
    if i>=2:
        ax[i].set_xlim(593e3, 603e3)
        ax[i].set_ylim(5188e3, 5196e3)
        ax[i].set_xticks(np.arange(594e3, 603e3, step=2e3))
        ax[i].set_yticks(np.arange(5188e3, 5197e3, step=2e3))
    else:
        ax[i].set_xlim(535e3, 541e3)
        ax[i].set_ylim(6468e3, 6475e3)
        ax[i].set_xticks(np.arange(536e3, 541e3, step=2e3))
        ax[i].set_yticks(np.arange(6468e3, 6475e3, step=2e3))
    # change labels from m to km
    ax[i].set_xticklabels([str(int(x/1e3)) for x in ax[i].get_xticks()])
    ax[i].set_yticklabels([str(int(x/1e3)) for x in ax[i].get_yticks()])
        
    # add text labels
    ax[i].text((ax[i].get_xlim()[1] - ax[i].get_xlim()[0])*0.05 + ax[i].get_xlim()[0],
               (ax[i].get_ylim()[1] - ax[i].get_ylim()[0])*0.9 + ax[i].get_ylim()[0],
               text_labels[i], backgroundcolor='w')
    

# add axis labels
ax[0].set_ylabel('Northing [km]')
ax[2].set_ylabel('Northing [km]')
ax[2].set_xlabel('Easting [km]')
ax[3].set_xlabel('Easting [km]')

# add legend
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc=(0.38, 0.92), ncols=2)

# fig.tight_layout()
plt.show()
    
# save figure
if save_figures:
    fig_fn = 'classification_performance_assessment_images.png'
    fig.savefig(out_path + fig_fn, facecolor='w', dpi=300, bbox_inches='tight')
    print('figure saved to file: ' + out_path + fig_fn)
    

## Figure 5. Median snowline elevations for the USGS Benchmark Glaciers

In [ ]:
# -----Settings and display parameters
site_names = ['Wolverine', 'Gulkana', 'LemonCreek', 'SouthCascade', 'Sperry']
site_names_display = ['Wolverine',  'Gulkana', 'Lemon Creek', 'South Cascade', 'Sperry']
text_labels = ['a)', 'b)', 'c)', 'd)', 'e)']

# -----Path to USGS mass balance data
usgs_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/GIS_data/USGS/benchmarkGlacier_massBalance/'

# -----Set up figures
# sall sites
fig, ax = plt.subplots(5, 1, figsize=(20, 24))
ax = ax.flatten()
plt.rcParams.update({'font.size':20, 'font.sans-serif':'Arial'})
fmt_month = matplotlib.dates.MonthLocator(bymonth=(5, 11)) # minor ticks every month
fmt_year = matplotlib.dates.YearLocator() # minor ticks every year
alpha = 0.9

# -----Loop through sites
for site_name, site_name_display, text_label, i in list(zip(site_names, site_names_display, text_labels, np.arange(0,len(site_names)))):
    
    print(site_name)
    
    # load estimated snow lines  
    sl_est_fns = glob.glob(study_sites_path + site_name + '/imagery/snowlines/*snowline.csv')
    sl_ests = gpd.GeoDataFrame()
    for sl_est_fn in sl_est_fns:
        sl_est = pd.read_csv(sl_est_fn)
        sl_ests = pd.concat([sl_ests, sl_est])
    sl_ests.reset_index(drop=True, inplace=True)
    sl_ests['datetime'] = sl_ests['datetime'].astype('datetime64[ns]')
    
    # define axis limits
    xmin, xmax = np.datetime64('2013-05-01T00:00:00'), np.datetime64('2022-12-01T00:00:00')
    sl_elev_median_min = np.nanmin(sl_ests['snowlines_elevs_median_m'])
    sl_elev_median_max = np.nanmax(sl_ests['snowlines_elevs_median_m'])
    ymin = sl_elev_median_min - 0.1*(sl_elev_median_max - sl_elev_median_min)
    ymax = sl_elev_median_max + 0.1*(sl_elev_median_max - sl_elev_median_min)
    
    # plot minimum elevation 
    ax[i].plot([xmin, xmax], [sl_elev_median_min, sl_elev_median_min], '--', color='grey')
    
    # plot
    # PlanetScope
    ax[i].plot(sl_ests['datetime'].loc[sl_ests['dataset']=='PlanetScope'], 
               sl_ests['snowlines_elevs_median_m'].loc[sl_ests['dataset']=='PlanetScope'], 
               '.', markeredgecolor='w', markerfacecolor=color_PlanetScope, 
               alpha=alpha, markersize=15, markeredgewidth=1, label='_nolegend_')
        
    # Sentinel-2 SR
    ax[i].plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Sentinel-2_SR'], 
               sl_ests['snowlines_elevs_median_m'].loc[sl_ests['dataset']=='Sentinel-2_SR'], 
               '*', markeredgecolor='w', markerfacecolor=color_Sentinel2, 
               alpha=alpha, markersize=15, markeredgewidth=1, label='_nolegend_')
    
    # Sentinel-2 TOA
    ax[i].plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Sentinel-2_TOA'], 
               sl_ests['snowlines_elevs_median_m'].loc[sl_ests['dataset']=='Sentinel-2_TOA'], 
               'o', markeredgecolor=color_Sentinel2, markerfacecolor='None', 
               alpha=alpha, markersize=5, markeredgewidth=3, label='_nolegend_')
    
    # Landsat
    ax[i].plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Landsat'], 
               sl_ests['snowlines_elevs_median_m'].loc[sl_ests['dataset']=='Landsat'], 
               '^', markeredgecolor='w', markerfacecolor=color_Landsat, 
               alpha=alpha, markersize=10, markeredgewidth=1, label='_nolegend_')                 
    
    # sett axis limits
    ax[i].set_xlim(xmin, xmax)
    ax[i].set_ylim(ymin, ymax)
    ax[i].grid()
    # x-labels
    ax[i].xaxis.set_minor_formatter(matplotlib.dates.DateFormatter('%b'))
    ax[i].xaxis.set_major_locator(fmt_month)
    ax[i].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%b'))
    sec_xaxis = ax[4].secondary_xaxis(-0.1)
    sec_xaxis.xaxis.set_major_locator(fmt_year)
    sec_xaxis.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y'))
    # Hide the second x-axis spines and ticks
    sec_xaxis.spines['bottom'].set_visible(False)
    sec_xaxis.tick_params(length=0, pad=10)
    if i<4:
        ax[i].set_xticklabels([])
        sec_xaxis.set_xticklabels([])
    # y-label on middle panel
    if i==2:
        ax[i].set_ylabel('Median snowline elevation [m]')
    # text label
    ax[i].text((xmax-xmin)*0.015 + xmin, (ymax-ymin)*0.87 + ymin, 
               text_label + ' ' + site_name_display, 
               bbox=dict(facecolor='white', edgecolor='black', pad=5))

    # -----Observed snow lines
    # define path to digitized snow lines
    sl_obs_path = study_sites_path + '../snowline-package/' + site_name + '/snowlines/'
    sl_obs_fns = glob.glob(sl_obs_path + '*.shp')
    # load DEM
    DEM_fn = glob.glob(study_sites_path + site_name + '/DEMs/*_clip.tif')[0]
    # load DEM as xarray DataSet
    DEM = xr.open_dataset(DEM_fn)
    DEM = DEM.rename({'band_data': 'elevation'})
    if len(np.shape(DEM.elevation.data))>2: # remove unnecessary dimensions
        DEM['elevation'] = DEM.elevation[0]
    # solve for optimal UTM zone
    DEM_WGS = DEM.rio.reproject('EPSG:4326')
    epsg_UTM = f.convert_wgs_to_utm(np.nanmean(DEM_WGS.x.data), np.nanmean(DEM_WGS.y.data))
    # reproject DEM
    DEM = DEM.rio.reproject('EPSG:'+epsg_UTM)
    # loop through observed snow lines
    for j, sl_obs_fn in enumerate(sl_obs_fns):
        # load observed snow line
        sl_obs = gpd.read_file(sl_obs_fn)
        # extract date from filename
        date = sl_obs_fn.split('/'+site_name+'_')[1][0:8]
        datetime = np.datetime64(date[0:4] + '-' + date[4:6] + '-' + date[6:8] + 'T00:00:00')
        # reproject snow line to UTM
        sl_obs_UTM = sl_obs.to_crs('EPSG:'+epsg_UTM)
        # interpolate elevation at snow line points
        if len(sl_obs_UTM) > 1:
            sl_obs_elev = np.array([DEM.sel(x=x, y=y, method='nearest').elevation.data 
                                for x, y in list(zip(sl_obs_UTM.geometry[1].xy[0], 
                                                     sl_obs_UTM.geometry[1].xy[1]))])
        else:
            sl_obs_elev = np.array([DEM.sel(x=x, y=y, method='nearest').elevation.data 
                                for x, y in list(zip(sl_obs_UTM.geometry[0].xy[0], 
                                                     sl_obs_UTM.geometry[0].xy[1]))])
        # calculate median snow line elevation
        sl_obs_elev_median = np.nanmedian(sl_obs_elev)
        # plot
        ax[i].plot(datetime, sl_obs_elev_median, 'xk', markersize=10, markeredgewidth=2, label='_nolegend_')   
            
    # load USGS ELA estimates
    usgs_fn = usgs_path + site_name + '/Output_' + site_name + '_Glacier_Wide_solutions_calibrated.csv'
    usgs_file = pd.read_csv(usgs_fn)
    ELA = usgs_file['ELA']
    ELA_date = usgs_file['Ba_Date'].astype('datetime64[ns]')
    ax[i].plot(ELA_date, ELA, 's', markerfacecolor='None', markeredgecolor='k', 
               ms=10, markeredgewidth=2, label='_nolegend_')
            
# -----Dummy points for legend
# observed
ax[0].plot(np.datetime64('1970-01-01'), 0, 'xk', 
           markersize=15, markeredgewidth=3, label='observed')
# USGS
ax[0].plot(np.datetime64('1970-01-01'), 0, 's', markerfacecolor='None', markeredgecolor='k', 
               ms=10, markeredgewidth=2, label='USGS ELA')
# Landsat
ax[0].plot(np.datetime64('1970-01-01'), 0, '^', 
           markeredgecolor=color_Landsat, markerfacecolor=color_Landsat, 
           markersize=12, label='Landsat 8/9')
# Sentinel-2 SR
ax[0].plot(np.datetime64('1970-01-01'), 0, '*',
           markeredgecolor=color_Sentinel2, markerfacecolor=color_Sentinel2, 
           markersize=18, label='Sentinel-2 SR')
# Sentinel-2 TOA
ax[0].plot(np.datetime64('1970-01-01'), 0, 'o',
           markeredgecolor=color_Sentinel2, markerfacecolor='w', 
           markersize=18, markeredgewidth=4, label='Sentinel-2 TOA')
# PlanetScope
ax[0].plot(np.datetime64('1970-01-01'), 0, '.', 
       markeredgecolor=color_PlanetScope, markerfacecolor=color_PlanetScope, 
       markersize=20, label='PlanetScope')
ax[0].legend(loc='center', bbox_to_anchor=(0.5, 1.15), ncol=6)

plt.show()
    
# -----Save figure
if save_figures:
    fig_fn = 'median_snowline_elevs.png'
    fig.savefig(out_path + fig_fn, facecolor='w', dpi=300)
    print('figure saved to file: ' + out_path + fig_fn)

In [ ]:
fig = plt.figure(figsize=(8,8))
col = plt.cm.viridis
for i, site_name in enumerate(site_names):
    usgs_fn = usgs_path + site_name+'/Output_'+site_name+'_Glacier_Wide_solutions_calibrated.csv'
    usgs_file = pd.read_csv(usgs_fn)
    ELA = usgs_file['ELA']
    ELA_date = usgs_file['Ba_Date'].astype('datetime64[ns]')
    plt.plot(ELA_date, ELA, '.-', color=col((i+1)/len(site_names)), label=site_name)
plt.legend()
plt.show()

## Figure 6. Example ideal and difficult conditions

In [ ]:
# -----Set up figure
fig, ax = plt.subplots(2, 2, figsize=(12, 12))
plt.rcParams.update({'font.size':28, 'font.sans-serif':'Arial'})
ax = ax.flatten()

# -----Ideal conditions
## PlanetScope - Wolverine - 20210815
# image
im_path = base_path + '../study-sites/Wolverine/imagery/PlanetScope/adjusted/'
im_fn = glob.glob(im_path + '*20210815*.nc')[0]
im = xr.open_dataset(im_fn)
im = im.isel(time=0)
# snowline
sl_fn = glob.glob(base_path + '../study-sites/Wolverine/imagery/PlanetScope/snowlines/*snowlines.pkl')[0]
sl = pd.read_pickle(sl_fn)
sl = sl.loc[sl['datetime']=='20210815T200000'].reset_index(drop=True)
# plot
ax[0].imshow(np.dstack([im['red'].data, im['green'].data, im['blue'].data]), 
             extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
ax[0].plot(*sl['snowlines_coords'][0].coords.xy, '.m', markersize=2)
ax[0].text(ax[0].get_xlim()[0] + 0.85*(ax[0].get_xlim()[1] - ax[0].get_xlim()[0]), 
           ax[0].get_ylim()[0] + 0.05*(ax[0].get_ylim()[1] - ax[0].get_ylim()[0]),
           'a)', bbox=dict(facecolor='white', edgecolor='black', pad=3))
ax[0].set_title('Ideal conditions')
ax[0].set_xticks([])
ax[0].set_yticks([])
## PlanetScope - Lemon Creek - 20220821
# image
im_path = base_path + '../study-sites/LemonCreek/imagery/PlanetScope/adjusted/'
im_fn = glob.glob(im_path + '*20220821*.nc')[0]
im = xr.open_dataset(im_fn)
im = im.isel(time=0)
# snowline
sl_fn = glob.glob(base_path + '../study-sites/LemonCreek/imagery/PlanetScope/snowlines/*snowlines.pkl')[0]
sl = pd.read_pickle(sl_fn)
sl = sl.loc[sl['datetime']=='20220821T190000'].reset_index(drop=True)
# plot
ax[2].imshow(np.dstack([im['red'].data, im['green'].data, im['blue'].data]), 
             extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
ax[2].plot(*sl['snowlines_coords'][0].coords.xy, '.m', markersize=2)
ax[2].text(ax[2].get_xlim()[0] + 0.8*(ax[2].get_xlim()[1] - ax[2].get_xlim()[0]), 
           ax[2].get_ylim()[0] + 0.05*(ax[2].get_ylim()[1] - ax[2].get_ylim()[0]),
           'c)', bbox=dict(facecolor='white', edgecolor='black', pad=3))
ax[2].set_xticks([])
ax[2].set_yticks([])

# -----Difficult conditions
## PlanetScope - Sperry - 20160816
# image
im_path = base_path + '../study-sites/Sperry/imagery/PlanetScope/adjusted/'
im_fn = glob.glob(im_path + '*20160816*.nc')[0]
im = xr.open_dataset(im_fn)
im = im.isel(time=0)
# snowline
sl_fn = glob.glob(base_path + '../study-sites/Sperry/imagery/PlanetScope/snowlines/*snowlines.pkl')[0]
sl = pd.read_pickle(sl_fn)
sl = sl.loc[sl['datetime']=='20160816T170000'].reset_index(drop=True)
# plot
ax[1].imshow(np.dstack([im['red'].data, im['green'].data, im['blue'].data]), 
             extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
ax[1].plot(*sl['snowlines_coords'][0].coords.xy, '.m', markersize=2)
ax[1].text(ax[1].get_xlim()[0] + 0.85*(ax[1].get_xlim()[1] - ax[1].get_xlim()[0]), 
           ax[1].get_ylim()[0] + 0.05*(ax[1].get_ylim()[1] - ax[1].get_ylim()[0]),
           'b)', bbox=dict(facecolor='white', edgecolor='black', pad=3))
ax[1].set_title('Difficult conditions')
ax[1].set_xticks([])
ax[1].set_yticks([])

## PlanetScope - Gulkana - 20170629
# image
im_path = base_path + '../study-sites/Gulkana/imagery/PlanetScope/adjusted/'
im_fn = glob.glob(im_path + '*20170629*.nc')[0]
im = xr.open_dataset(im_fn)
im = im.isel(time=0)
# snowline
sl_fn = glob.glob(base_path + '../study-sites/Gulkana/imagery/PlanetScope/snowlines/*snowlines.pkl')[0]
sl = pd.read_pickle(sl_fn)
sl = sl.loc[sl['datetime']=='20170629T200000'].reset_index(drop=True)
# plot
ax[3].imshow(np.dstack([im['red'].data, im['green'].data, im['blue'].data]), 
             extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
ax[3].plot(*sl['snowlines_coords'][0].coords.xy, '.m', markersize=2)
ax[3].text(ax[3].get_xlim()[0] + 0.9*(ax[3].get_xlim()[1] - ax[3].get_xlim()[0]), 
           ax[3].get_ylim()[0] + 0.08*(ax[3].get_ylim()[1] - ax[3].get_ylim()[0]),
           'd)', bbox=dict(facecolor='white', edgecolor='black', pad=3))
ax[3].set_xticks([])
ax[3].set_yticks([])

fig.tight_layout()
plt.show()

# -----Save figure
fig_fn = 'example_ideal+difficult_conditions.png'
fig.savefig(out_path + fig_fn, dpi=300, facecolor='w')
print('figure saved to file: '+ out_path + fig_fn)

## Snow cover products comparison

In [ ]:
# # -----Load Landsat fSCA
# LS_fn = base_path+'../study-sites/Wolverine/imagery/Landsat/fSCA/LC08_AK_016008_20210829_20210913_02_SNOW/LC08_AK_016008_20210829_20210913_02_VIEWABLE_SNOW_UTM.TIF'
# LS = rxr.open_rasterio(LS_fn)
# # remove no-data values
# LS = LS.where(LS != -9999)
# # account for image multiplier
# LS_scalar = 0.001
# LS = LS * LS_scalar
# crs = LS.rio.crs.to_string()

# # -----Load MODIS fSCA
# M_fn = base_path+'../study-sites/Wolverine/imagery/MODIS/Terra_fSCA/2021_08_15.tif'
# M = rxr.open_rasterio(M_fn)
# # grab snow cover band
# M_fSCA = M.isel(band=0)
# # remove no data values
# M_fSCA = M_fSCA.where(M_fSCA != -3.2768e04)
# # reproject 
# M_fSCA= M_fSCA.rio.reproject(crs)

# # -----Load PlanetScope image and snow
# # RGB image
# PS_path = base_path+'../study-sites/Wolverine/imagery/PlanetScope/adjusted-filtered/'
# PS_fn = '20210815_20_adj.tif'
# PS = rxr.open_rasterio(PS_path + PS_fn)
# PS = PS / 1e4
# # classify image
# clf_fn = base_path+'/inputs-outputs/PS_classifier_all_sites.sav'
# clf = pickle.load(open(clf_fn, 'rb'))
# feature_cols_fn = base_path+'inputs-outputs/PS_feature_cols.pkl'
# feature_cols = pickle.load(open(feature_cols_fn,'rb'))
# sys.path.insert(1, base_path+'functions/')
# from ps_pipeline_utils import classify_image
# im_classified_fn, im = classify_image(PS_fn, PS_path, clf, feature_cols, False, None, out_path)
# # load classified image
# im_classified = rxr.open_rasterio(out_path + im_classified_fn) 

In [ ]:
# # -----Create snow colormap
# color_snow = '#4eb3d3'
# color_no_snow = 'w'
# # create colormap
# colors = [color_no_snow, color_snow]
# cmp = cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

# # -----Plot
# fig, ax = plt.subplots(2, 2, figsize=(10,10))
# ax = ax.flatten()
# plt.rcParams.update({'font.size':16, 'font.sans-serif':'Arial'})
# xmin, xmax, ymin, ymax = 391, 399, 6694, 6702
# # MODIS
# M_im = ax[0].imshow(M_fSCA.data, cmap=cmp, clim=(0,100),
#                     extent=(np.min(M_fSCA.x.data)/1000, np.max(M_fSCA.x.data)/1000, 
#                             np.min(M_fSCA.y.data)/1000, np.max(M_fSCA.y.data)/1000))
# ax[0].set_xticks(np.linspace(392, 398, num=4))
# ax[0].set_yticks(np.linspace(6694, 6702, num=5))
# ax[0].set_xticklabels([])
# ax[0].set_xlim(xmin, xmax)
# ax[0].set_ylim(ymin, ymax)
# ax[0].set_ylabel('Northing [km]')
# ax[0].set_title('a) MODIS f$_{SCA}$')
# # LS
# LS_im = ax[1].imshow(LS_fSCA, cmap=cmp, clim=(0,1),
#                    extent=(np.min(LS_x)/1000, np.max(LS_x)/1000, np.min(LS_y)/1000, np.max(LS_y)/1000))
# ax[1].set_xticks(np.linspace(392, 398, num=4))
# ax[1].set_yticks(np.linspace(6694, 6702, num=5))
# ax[1].set_xticklabels([])
# ax[1].set_yticklabels([])
# ax[1].set_xlim(xmin, xmax)
# ax[1].set_ylim(ymin, ymax)
# ax[1].set_title('b) Landsat 8 f$_{SCA}$')
# # PS RGB
# ax[2].imshow(np.dstack([PS.data[2], PS.data[1], PS.data[0]]),
#            extent=(np.min(PS.x.data)/1000, np.max(PS.x.data)/1000, np.min(PS.y.data)/1000, np.max(PS.y.data)/1000))
# ax[2].set_xticks(np.linspace(392, 398, num=4))
# ax[2].set_yticks(np.linspace(6694, 6702, num=5))
# ax[2].set_xlim(xmin, xmax)
# ax[2].set_ylim(ymin, ymax)
# ax[2].set_ylabel('Northing [km]')
# ax[2].set_xlabel('Easting [km]')
# ax[2].set_title('c) PlanetScope RGB')
# # PS snow
# im_classified = im_classified.where(im_classified!=-9999)
# im_binary = xr.where(im_classified<=2, 1, 0)
# PS_snow_im = ax[3].imshow(im_binary.data[0], cmap=cmp, clim=(0,1),
#                    extent=(np.min(PS.x.data)/1000, np.max(PS.x.data)/1000, np.min(PS.y.data)/1000, np.max(PS.y.data)/1000))
# ax[3].set_xticks(np.linspace(392, 398, num=4))
# ax[3].set_yticks(np.linspace(6694, 6702, num=5))
# ax[3].set_yticklabels([])
# ax[3].set_xlim(xmin, xmax)
# ax[3].set_ylim(ymin, ymax)
# ax[3].set_xlabel('Easting [km]')
# ax[3].set_title('d) PlanetScope SCA')
# # colorbar
# cbar_ax = fig.add_axes([0.92, 0.35, 0.02, 0.3])
# fig.colorbar(M_im, cax=cbar_ax)
# plt.show()

# if save_figures:
#     fig.savefig(out_path+'comparing_SCA_products.png', dpi=300, facecolor='white', edgecolor='none')
#     print('figure saved to file')

## Study sites: RGI regions 1 and 2 (Alaska, the Western U.S. and Canada)

In [ ]:
# -----Define paths in directory
# path to RGI data
RGI_path = '/Volumes/GoogleDrive/My Drive/Research/PhD/GIS_data/RGI/'
# RGI shapefile names
RGI_fns = ['01_rgi60_Alaska/01_rgi60_Alaska.shp', 
           '02_rgi60_WesternCanadaUS/02_rgi60_WesternCanadaUS.shp']

# -----Load, format, filter, plot RGI glacier outlines
# Create geopandas.DataFrame for storing RGIs
RGI = gpd.GeoDataFrame()
# Read RGI files
for RGI_fn in RGI_fns:
    file = gpd.read_file(RGI_path + RGI_fn)
    RGI = pd.concat([RGI, file])
# subset to glaciers with area > 5 km^2
RGI_gt5 = RGI.loc[RGI['Area'] > 5].reset_index(drop=True)
# change int data types to float for saving
RGI_gt5[['Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 'Status', 'Connect', 
         'Form', 'TermType', 'Surging', 'Linkages']] = RGI_gt5[['Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 
                                                            'Status', 'Connect', 'Form', 'TermType', 'Surging', 'Linkages']].astype(float)

# -----Grab list of all unique regions and subregions in dataset
regions_subregions = sorted(RGI_gt5[['O1Region', 'O2Region']].drop_duplicates().values,
                            key=operator.itemgetter(0, 1))
subregions_names = ['Brooks Range', 'Alaska Range', 'Aleutians', 'W. Chugach Mtns.', 'St. Elias Mtns.', 
                    'N. Coast Ranges', 'N. Rockies', 'N. Cascades', 'S. Rockies', 'S. Cascades']
subregions_colors = ['c', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', 
                     '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a']

# -----Plot all sites with color distinguishing subregions
fig, ax = plt.subplots(1, 1, figsize=(12,10))
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})
crs = 'EPSG:9822' # Albers Equal Conic projection
i=0
for region, subregion in regions_subregions:
    RGI_gt5_subregion = RGI_gt5.loc[(RGI_gt5['O1Region']==region) & (RGI_gt5['O2Region']==subregion)]
    RGI_gt5_subregion_reproj = RGI_gt5_subregion.to_crs(crs)
    for j in range(0, len(RGI_gt5_subregion)):
        polygon = RGI_gt5_subregion_reproj.iloc[j]['geometry']
        if j==0:
            label=subregions_names[i]
        else:
            label='_nolegend_'
        ax.plot(*polygon.exterior.xy, label=label, color=subregions_colors[i])
    i+=1
cx.add_basemap(ax, crs=crs, source=cx.providers.Esri.WorldShadedRelief, attribution=False)
ax.legend(loc='center right', title='RGI Subregions', bbox_to_anchor=[1.25, 0.5, 0.2, 0.2])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.grid()
plt.show()

# -----Save figure
fig.savefig(out_path + 'RGI_regions_1+2.png', dpi=300, facecolor='w')
print('figure saved to file')

## Median snow line elevations for individual sites

In [ ]:
# -----Settings and display parameters
site_name = 'Gulkana' # as shown in folder name
# site_name_display = 'Gulkana' # how to display on figure
dataset_colors = {'Landsat': '#33a02c',
                  'Sentinel2_TOA': '#1f78b4',
                  'Sentinel2_SR': '#1f78b4',
                  'PlanetScope': '#a6cee3'
                 }

# -----Path to USGS mass balance data
usgs_path = '/Volumes/GoogleDrive/My Drive/Research/PhD/GIS_data/USGS/benchmarkGlacier_massBalance/'
    
# -----Load estimated snow lines  
sl_est_fns = glob.glob(base_path + '../study-sites/' + site_name + '/imagery/snowlines/*snowline.pkl')
sl_ests = pd.DataFrame()
for sl_est_fn in sl_est_fns:
    sl_est = pd.read_pickle(sl_est_fn)
    sl_ests = pd.concat([sl_ests, sl_est])
sl_ests = sl_ests.reset_index(drop=True)
sl_ests['datetime'] = sl_ests['datetime'].astype(np.datetime64)

# -----Plot
# Set up figure
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
plt.rcParams.update({'font.size':16, 'font.sans-serif':'Arial'})
fmt_month = matplotlib.dates.MonthLocator(bymonth=(5, 11)) # minor ticks every month
fmt_year = matplotlib.dates.YearLocator() # minor ticks every year
# PlanetScope
ax.plot(sl_ests['datetime'].loc[sl_ests['dataset']=='PlanetScope'], 
           sl_ests['snowlines_elevs_median'].loc[sl_ests['dataset']=='PlanetScope'], 
           '.', markeredgecolor='w', markerfacecolor=dataset_colors['PlanetScope'], 
           markersize=20, markeredgewidth=1, label='_nolegend_')
# Sentinel-2 TOA
ax.plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Sentinel2_TOA'], 
           sl_ests['snowlines_elevs_median'].loc[sl_ests['dataset']=='Sentinel2_TOA'], 
           '*', markeredgecolor='w', markerfacecolor=dataset_colors['Sentinel2_TOA'], 
           markersize=20, markeredgewidth=1, label='_nolegend_')
# Sentinel-2 SR
ax.plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Sentinel2_SR'], 
           sl_ests['snowlines_elevs_median'].loc[sl_ests['dataset']=='Sentinel2_SR'], 
           '*', markeredgecolor=dataset_colors['Sentinel2_SR'], markerfacecolor='None', 
           markersize=20, markeredgewidth=1, label='_nolegend_')
# Landsat
ax.plot(sl_ests['datetime'].loc[sl_ests['dataset']=='Landsat'], 
           sl_ests['snowlines_elevs_median'].loc[sl_ests['dataset']=='Landsat'], 
           '^', markeredgecolor='w', markerfacecolor=dataset_colors['Landsat'], 
           markersize=15, markeredgewidth=1, label='_nolegend_')                
        
# -----Dummy points for legend
# observed
ax.plot(np.datetime64('1970-01-01'), 0, 'xk', 
           markersize=15, markeredgewidth=3, label='observed')
# USGS
ax.plot(np.datetime64('1970-01-01'), 0, 's', markerfacecolor='None', markeredgecolor='r', 
               ms=10, markeredgewidth=2, label='USGS ELA')
# Landsat
ax.plot(np.datetime64('1970-01-01'), 0, '^', 
           markeredgecolor=dataset_colors['Landsat'], markerfacecolor=dataset_colors['Landsat'], 
           markersize=12, label='Landsat 8/9')
# Sentinel-2 TOA
ax.plot(np.datetime64('1970-01-01'), 0, '*',
           markeredgecolor='w', markerfacecolor=dataset_colors['Sentinel2_TOA'], 
           markersize=18, label='Sentinel-2 TOA')
# Sentinel-2 SR
ax.plot(np.datetime64('1970-01-01'), 0, '*',
           markeredgecolor=dataset_colors['Sentinel2_SR'], markerfacecolor='None', 
           markersize=18, label='Sentinel-2 SR')
# PlanetScope
ax.plot(np.datetime64('1970-01-01'), 0, '.', 
       markeredgecolor=dataset_colors['PlanetScope'], markerfacecolor=dataset_colors['PlanetScope'], 
       markersize=20, label='PlanetScope')
ax.legend(loc='center', bbox_to_anchor=(0.5, 1.05), ncol=6)

# -----Observed snow lines
# define path to digitized snow lines
sl_obs_path = base_path + '../snowline-package/' + site_name + '/snowlines/'
sl_obs_fns = glob.glob(sl_obs_path + '*.shp')
# load AOI as gpd.GeoDataFrame
AOI_fn = base_path + '../study-sites/' + site_name + '/glacier_outlines/' + site_name + '_USGS_*.shp'
AOI_fn = glob.glob(AOI_fn)[0]
AOI = gpd.read_file(AOI_fn)
# load DEM from GEE
DEM, AOI_UTM = pf.query_GEE_for_DEM(AOI)
# loop through observed snow lines
for j, sl_obs_fn in enumerate(sl_obs_fns):
    # load observed snow line
    sl_obs = gpd.read_file(sl_obs_fn)
    # extract date from filename
    date = sl_obs_fn.split('/'+site_name+'_')[1][0:8]
    datetime = np.datetime64(date[0:4] + '-' + date[4:6] + '-' + date[6:8]
                             + 'T00:00:00')
    # reproject snow line to UTM
    sl_obs_UTM = sl_obs.to_crs(str(AOI_UTM.crs.to_epsg()))
    # interpolate elevation at snow line points
    if len(sl_obs_UTM) > 1:
        sl_obs_elev = np.array([DEM.sel(time=DEM.time.data[0], x=x, y=y, method='nearest').elevation.data 
                            for x, y in list(zip(sl_obs_UTM.geometry[1].xy[0], 
                                                 sl_obs_UTM.geometry[1].xy[1]))])
    else:
        sl_obs_elev = np.array([DEM.sel(time=DEM.time.data[0], x=x, y=y, method='nearest').elevation.data 
                            for x, y in list(zip(sl_obs_UTM.geometry[0].xy[0], 
                                                 sl_obs_UTM.geometry[0].xy[1]))])
    # calculate median snow line elevation
    sl_obs_elev_median = np.nanmedian(sl_obs_elev)
    # plot
    ax.plot(datetime, sl_obs_elev_median, 'xk', markersize=10, markeredgewidth=2, label='_nolegend_')   
            
    # load USGS ELA estimates
    usgs_fn = usgs_path + site_name+'/Output_'+site_name+'_Glacier_Wide_solutions_calibrated.csv'
    usgs_file = pd.read_csv(usgs_fn)
    ELA = usgs_file['ELA']
    ELA_date = usgs_file['Ba_Date'].astype(np.datetime64)
    ax.plot(ELA_date, ELA, 's', markerfacecolor='None', markeredgecolor='r', 
               ms=10, markeredgewidth=2, label='_nolegend_')

# -----Adjust axes
# axis limits
xmin, xmax = np.datetime64('2017-05-01T00:00:00'), np.datetime64('2023-01-01T00:00:00')
sl_elev_median_min = np.min(sl_ests['snowlines_elevs_median'])
sl_elev_median_max = np.max(sl_ests['snowlines_elevs_median'])
ymin = sl_elev_median_min - 0.1*(sl_elev_median_max - sl_elev_median_min)
ymax = sl_elev_median_max + 0.1*(sl_elev_median_max - sl_elev_median_min)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.grid()
# x-labels
ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter('%b'))
ax.xaxis.set_major_locator(fmt_month)
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%b'))
sec_xaxis = ax.secondary_xaxis(-0.1)
sec_xaxis.xaxis.set_major_locator(fmt_year)
sec_xaxis.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y'))
# Hide the second x-axis spines and ticks
sec_xaxis.spines['bottom'].set_visible(False)
sec_xaxis.tick_params(length=0, pad=10)
# y-label
ax.set_ylabel('Elevation [m]')
plt.show()
    
# -----Save figure
fig_fn = 'median_snowline_elevs_' + site_name + '.png'
fig.savefig(out_path + fig_fn, facecolor='w', dpi=300)
print('figure saved to file: '+out_path+fig_fn)

## Training data characteristics by dataset

In [ ]:
# -----Load dataset dictionary
with open(base_path + 'inputs-outputs/datasets_characteristics.pkl', 'rb') as fn:
    dataset_dict = pickle.load(fn)

# -----Define band ranges
L8_dict = dataset_dict['Landsat']
L8_dict['bands'] = {'SR_B2': {'name': 'blue',
                              'min_nm': 450,
                              'max_nm': 510},
                    'SR_B3': {'name': 'green',
                              'min_nm': 530,
                              'max_nm': 590},
                    'SR_B4': {'name': 'red',
                              'min_nm': 640,
                              'max_nm': 670},
                    'SR_B5': {'name': 'NIR',
                              'min_nm': 850,
                              'max_nm': 880},
                    'SR_B6': {'name': 'SWIR1',
                              'min_nm': 1570,
                              'max_nm': 1650},
                    'SR_B7': {'name': 'SWIR2',
                              'min_nm': 2110,
                              'max_nm': 2290}
                   }

S2_dict = dataset_dict['Sentinel-2']
S2_dict['bands'] = {'B2': {'name': 'blue',
                           'wavelength_min_nm': 459,
                           'wavelength_max_nm': 525
                          },
                    'B3': {'name': 'green',
                           'wavelength_min_nm': 541,
                           'wavelength_max_nm': 577
                          },
                    'B4': {'name': 'red',
                           'wavelength_min_nm': 649,
                           'wavelength_max_nm': 680
                          },
                    'B8': {'name': 'NIR',
                           'wavelength_min_nm': 780,
                           'wavelength_max_nm': 886
                          },
                    'B11': {'name': 'SWIR1',
                           'wavelength_min_nm': 1567,
                           'wavelength_max_nm': 1658
                            
                          },
                    'B12': {'name': 'SWIR2',
                           'wavelength_min_nm': 2114,
                           'wavelength_max_nm': 2289
                          }
                   }
PS_dict = dataset_dict['PlanetScope']
PS_dict['bands'] = {'B1': {'name': 'blue',
                           'wavelength_min_nm':,
                           'wavelength_max_nm': 
                          }
